In [5]:
!pip install torch torchvision Pillow scikit-learn matplotlib numpy torchsummary


In [13]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from torchsummary import summary

In [14]:
class DiabeticRetinopathyDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir  # Path to dataset folder
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # Loop through each class (0-4) and collect image paths and labels
        for label in range(5):  # 0-4 labels (diabetic retinopathy stages)
            class_folder = os.path.join(data_dir, str(label))  # Folder named 0, 1, 2, 3, 4
            for img_name in os.listdir(class_folder):
                if img_name.endswith(('.jpg', '.png', '.jpeg')):  # Check image formats
                    self.image_paths.append(os.path.join(class_folder, img_name))
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Load image
        img = Image.open(self.image_paths[idx])
        label = self.labels[idx]  # Get the corresponding label

        # Apply transformations (resize, tensor conversion, normalization)
        if self.transform:
            img = self.transform(img)

        return img, label


In [15]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 (required for ResNet)
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet stats
])

In [16]:
data_dir = r"C:\Users\sneha\Downloads\colored_images"

# Create dataset instance
dataset = DiabeticRetinopathyDataset(data_dir, transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create DataLoader for training and validation datasets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [17]:
model = models.resnet18(pretrained=True)  # Load pre-trained ResNet18
model.fc = nn.Linear(model.fc.in_features, 5)

C:\Users\sneha\AppData\Roaming\Python\Python313\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\sneha\AppData\Roaming\Python\Python313\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\sneha/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100.0%


In [18]:
summary(model, (3, 224, 224)) 

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [19]:
criterion = nn.CrossEntropyLoss()  # Cross entropy loss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001) 

In [20]:
def imshow(img):
    img = img / 2 + 0.5  # Unnormalize the image
    npimg = img.numpy()  # Convert to numpy array
    plt.imshow(np.transpose(npimg, (1, 2, 0)))  # Convert from (C, H, W) to (H, W, C)
    plt.show()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 10  # Set number of epochs

train_losses = []
train_accuracies = []
val_accuracies = []

# Train the model for multiple epochs
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0

    # Start training over batches
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Clear the previous gradients

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

    # Calculate and store training loss and accuracy
    train_loss = running_loss / len(train_loader)
    train_accuracy = correct_preds / total_preds * 100
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # Print epoch training details
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    val_preds = []
    val_true = []

    with torch.no_grad():  # No gradient computation during validation
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            val_preds.extend(predicted.cpu().numpy())
            val_true.extend(labels.cpu().numpy())

    # Calculate and store validation accuracy
    val_accuracy = accuracy_score(val_true, val_preds)
    val_accuracies.append(val_accuracy)

    print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

# After training is complete, plot the summary graphs
def plot_training_summary():
    epochs = range(1, num_epochs + 1)

    # Plot Training Loss
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label='Train Loss')
    plt.title('Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    # Plot Training Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label='Train Accuracy')
    plt.title('Training Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Display the plots
    plt.show()

# Call the function to plot graphs after training
plot_training_summary()

In [ ]:
import matplotlib.pyplot as plt

# Collect training losses and accuracies
train_losses = []
train_accuracies = []
val_accuracies = []

# Train the model for multiple epochs
num_epochs = 10  # Set number of epochs

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0

    # Start training over batches
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Clear the previous gradients

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

    # Calculate and store training loss and accuracy
    train_loss = running_loss / len(train_loader)
    train_accuracy = correct_preds / total_preds * 100
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # Print epoch training details
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    val_preds = []
    val_true = []

    with torch.no_grad():  # No gradient computation during validation
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            val_preds.extend(predicted.cpu().numpy())
            val_true.extend(labels.cpu().numpy())

    # Calculate and store validation accuracy
    val_accuracy = accuracy_score(val_true, val_preds)
    val_accuracies.append(val_accuracy)

    print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

# Plotting the training history and accuracy

def plot_training_history():
    epochs = range(1, num_epochs + 1)

    # Plot Training Loss
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label='Train Loss')
    plt.title('Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)

    # Plot Training Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label='Train Accuracy')
    plt.title('Training Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.grid(True)

    # Display the plots
    plt.tight_layout()
    plt.show()

# After training, plot the training history and accuracy graphs
plot_training_history()


Epoch 1/10
Train Loss: 0.6160, Train Accuracy: 76.58%
Validation Accuracy: 72.58%

Epoch 2/10
Train Loss: 0.5362, Train Accuracy: 79.96%
Validation Accuracy: 79.81%

Epoch 3/10
Train Loss: 0.4694, Train Accuracy: 82.38%
Validation Accuracy: 80.08%
